In [1]:
import pandas as pd
import sqlite3
import os
import tkinter as tk
from tkinter import filedialog
import xml.etree.ElementTree as ET
from datetime import datetime

# Ask user to select folder containing the Session XML file
root = tk.Tk()
root.withdraw()  # Hide the root window
selected_folder = filedialog.askdirectory(initialdir='D:/Pro-Sup Test/Data/')
if not selected_folder:
    raise ValueError("No folder was selected.")

# Extract the test_date from the selected folder name
folder_name = os.path.basename(selected_folder)
test_date = folder_name.split('_', 1)[0]  # Extract '2024-08-13' from '2024-08-13_105_Growth Plate_'

# Find the XML file titled "Session" in the selected folder
xml_file_path = ''
for r, dirs, files in os.walk(selected_folder):
    for file in files:
        if file.lower().startswith('session') and file.lower().endswith('.xml'):
            xml_file_path = os.path.join(r, file)
            break
    if xml_file_path:
        break

if not xml_file_path:
    raise FileNotFoundError("No 'Session' XML file found in the selected folder.")

# Parse the XML file
tree = ET.parse(xml_file_path)
root_xml = tree.getroot()

# Extract required fields from XML
name = root_xml.find(".//Name").text
dob = root_xml.find(".//DOB").text
height = root_xml.find(".//Height").text
weight = root_xml.find(".//Weight").text
injury_history = root_xml.find(".//Injury_History").text
season_phase = root_xml.find(".//Season_Phase").text
dynomometer_score = root_xml.find(".//Dynomometer_Score").text
comments = root_xml.find(".//Comments").text

# Calculate age from DOB
dob_date = datetime.strptime(dob, "%Y-%m-%d")
today = datetime.today()
age = today.year - dob_date.year - ((today.month, today.day) < (dob_date.month, dob_date.day))

# Define final columns
final_columns = [
    'name', 'test_date', 'age', 'height', 'weight', 'injury_history', 'season_phase', 'dynomometer_score', 'comments',
    'forearm_rom_0to10', 'forearm_rom_10to20', 'forearm_rom_20to30', 'forearm_rom',
    'tot_rom_0to10', 'tot_rom_10to20', 'tot_rom_20to30', 'tot_rom',
    'num_of_flips_0_10', 'num_of_flips_10_20', 'num_of_flips_20_30', 'num_of_flips',
    'avg_velo_0_10', 'avg_velo_10_20', 'avg_velo_20_30', 'avg_velo'
]

# Create a DataFrame with XML data and NULL for ASCII columns
data = {
    'name': name,
    'test_date': test_date,
    'age': age,
    'height': height,
    'weight': weight,
    'injury_history': injury_history,
    'season_phase': season_phase,
    'dynomometer_score': dynomometer_score,
    'comments': comments,
    'forearm_rom_0to10': None,
    'forearm_rom_10to20': None,
    'forearm_rom_20to30': None,
    'forearm_rom': None,
    'tot_rom_0to10': None,
    'tot_rom_10to20': None,
    'tot_rom_20to30': None,
    'tot_rom': None,
    'num_of_flips_0_10': None,
    'num_of_flips_10_20': None,
    'num_of_flips_20_30': None,
    'num_of_flips': None,
    'avg_velo_0_10': None,
    'avg_velo_10_20': None,
    'avg_velo_20_30': None,
    'avg_velo': None
}

df = pd.DataFrame([data], columns=final_columns)

# Connect to the SQLite database and create table if needed
db_path = 'D:/Pro-Sup Test/pro-sup_data.sqlite'
conn = sqlite3.connect(db_path)

# Ensure table exists
df.head(0).to_sql('pro_sup_data', conn, if_exists='append', index=False)

# Insert the row with XML data and NULL ASCII columns
df.to_sql('pro_sup_data', conn, if_exists='append', index=False)

conn.close()

# In the cell that pulls the XML/ASCII
global NEW_NAME, NEW_TEST_DATE
NEW_NAME = root_xml.find(".//Name").text
NEW_TEST_DATE = folder_name.split('_', 1)[0]

print("Globals set:", NEW_NAME, NEW_TEST_DATE)

print(f"XML data inserted for {name} with test date {test_date}.")


XML data inserted for Parnell, Josh with test date 2025-01-08.


In [2]:
import re
import pandas as pd
import sqlite3


def extract_test_date_from_ascii(ascii_file_path: str) -> str:
    """
    Extracts the test date in 'YYYY-MM-DD' format from the first file path in the ASCII file.
    
    Args:
        ascii_file_path (str): Path to the ASCII file.
    
    Returns:
        str: Extracted test date in 'YYYY-MM-DD' format.
    """
    with open(ascii_file_path, 'r') as file:
        lines = file.readlines()
        # Extract the first file path
        first_file_path = lines[0].strip().split('\t')[0]
        # Split the path to navigate the folder structure
        parts = first_file_path.split('\\')  # Split by folder structure
        if len(parts) > 4:  # Ensure we have enough subfolders
            date_folder = parts[4]  # Get the folder containing the date
            # Use regex to extract 'YYYY-MM-DD' format
            match = re.match(r'^\d{4}-\d{2}-\d{2}', date_folder)
            if match:
                return match.group(0)  # Return the matched date
            else:
                raise ValueError(f"Unable to extract test date from folder: {date_folder}")
        else:
            raise ValueError("Unexpected file path structure: Unable to extract test date.")


# Path to ASCII file
text_file_path = 'D:/Pro-Sup Test/pro-sup_data.txt'

# Dynamically extract the test date from the ASCII file
test_date = extract_test_date_from_ascii(text_file_path)
print(f"Extracted test date: {test_date}")

with open(text_file_path, 'r') as file:
    lines = file.readlines()
    data = [line.strip().split('\t') for line in lines]

# Extract header and data rows
header = data[1]
df_data = [row[1:] for row in data[5:]]
df_ascii = pd.DataFrame(df_data, columns=header)

# Replace hyphens with underscores
df_ascii.columns = [col.replace('-', '_') for col in df_ascii.columns]

row_ascii = df_ascii.iloc[0]

# Extract ASCII data
values = {col: row_ascii.get(col) for col in df_ascii.columns}

# Construct the UPDATE statement
set_clause = ", ".join([f"{col} = ?" for col in values.keys()])
params = list(values.values()) + [name, test_date]

update_sql = f"UPDATE pro_sup_data SET {set_clause} WHERE name = ? AND test_date = ?;"

conn = sqlite3.connect('D:/Pro-Sup Test/pro-sup_data.sqlite')
cursor = conn.cursor()
cursor.execute(update_sql, params)
conn.commit()
conn.close()

print(f"ASCII data updated for {name} on test date {test_date}.")

ascii_file_path = 'D:/Pro-Sup Test/pro-sup_data.txt'
test_date = extract_test_date_from_ascii(ascii_file_path)
print(f"Extracted test date: {test_date}")


Extracted test date: 2025-01-08
ASCII data updated for Parnell, Josh on test date 2025-01-08.
Extracted test date: 2025-01-08


In [5]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd
import sqlite3
import numpy as np

# Replace these with the actual name and test_date
# for the participant you just inserted.
# (Or dynamically pass them in from your other script.)
NEW_PARTICIPANT_NAME = "John Doe"
NEW_PARTICIPANT_TEST_DATE = "2024-08-13"

# Path to your SQLite database
DB_PATH = "D:/Pro-Sup Test/pro-sup_data.sqlite"

app = dash.Dash(__name__)

def get_database_df(db_path=DB_PATH):
    """
    Pull all data from the pro_sup_data table.
    """
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query("SELECT * FROM pro_sup_data", conn)
    conn.close()
    return df

def build_comparison_figure(name, test_date):
    """
    Build a Plotly figure comparing the newly inserted participant's
    data to the rest of the participants' data, focusing on columns
    containing 'tot_rom_', 'forearm_rom_', and 'num_of_flips_'.
    """

    # Read everything from the DB
    df = get_database_df(DB_PATH)

    # Identify columns of interest
    columns_of_interest = [
        col for col in df.columns 
        if ("tot_rom_" in col) or ("forearm_rom_" in col) or ("num_of_flips_" in col)
    ]

    # Separate the newly inserted row from the rest
    df_new = df[(df["name"] == name) & (df["test_date"] == test_date)]
    df_others = df[(df["name"] != name) | (df["test_date"] != test_date)]

    # If for some reason there is more than one new row, just take the first
    if len(df_new) == 0:
        raise ValueError(f"No matching row found for name={name} and test_date={test_date}.")
    if len(df_new) > 1:
        df_new = df_new.iloc[[0]]

    # Convert columns to numeric (in case they are stored as strings)
    for c in columns_of_interest:
        df_new[c] = pd.to_numeric(df_new[c], errors='coerce')
        df_others[c] = pd.to_numeric(df_others[c], errors='coerce')

    # Extract the new participant's values as a list
    new_values = [df_new.iloc[0][col] for col in columns_of_interest]

    # Compute the mean of all other participants for each column
    others_means = [df_others[col].mean() for col in columns_of_interest]

    # Build a small DataFrame for plotting
    plot_df = pd.DataFrame({
        "Column": columns_of_interest * 2,
        "Value": new_values + others_means,
        "Group": ["New Participant"] * len(columns_of_interest) + ["Others (Avg)"] * len(columns_of_interest)
    })

    # Create a grouped bar chart
    fig = px.bar(
        plot_df, 
        x="Column", 
        y="Value", 
        color="Group", 
        barmode="group", 
        title=f"Comparison for {name} ({test_date})"
    )
    fig.update_layout(xaxis_title="Column of Interest", yaxis_title="Value")

    return fig

# Build layout
app.layout = html.Div([
    html.H1("Pro-Sup Data Comparison Report"),
    dcc.Graph(id="comparison-graph")
])

# Use a simple callback or just build the figure once
@app.callback(
    dash.dependencies.Output("comparison-graph", "figure"),
    [dash.dependencies.Input("comparison-graph", "id")]  # Dummy input to trigger once
)
def update_graph(_):
    return build_comparison_figure(
        name=NEW_PARTICIPANT_NAME,
        test_date=NEW_PARTICIPANT_TEST_DATE
    )

if __name__ == "__main__":
    # Run in debug mode. Access it at http://127.0.0.1:8052
    app.run_server(debug=True)


OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.

In [3]:
# Reorders the database to be in alphabetical order

import sqlite3


db_path = "D:/Pro-Sup Test/pro-sup_data.sqlite" 
sort_column = "name"     

def reorder_all_tables(db_path, sort_column):
    try:
        # Connect to the database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Fetch all table names in the database
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()

        for table in tables:
            table_name = table[0]

            # Skip system tables like sqlite_sequence
            if table_name.startswith("sqlite_"):
                continue

            print(f"Processing table: {table_name}")

            # Check if the column exists in the current table
            cursor.execute(f"PRAGMA table_info({table_name});")
            columns = [info[1] for info in cursor.fetchall()]
            if sort_column not in columns:
                print(f"Skipping table '{table_name}' - Column '{sort_column}' not found.")
                continue

            # Create a new sorted table
            temp_table = f"{table_name}_sorted"
            cursor.execute(f"CREATE TABLE {temp_table} AS SELECT * FROM {table_name} ORDER BY {sort_column} ASC;")
            
            # Drop the old table
            cursor.execute(f"DROP TABLE {table_name};")
            
            # Rename the new table to the original name
            cursor.execute(f"ALTER TABLE {temp_table} RENAME TO {table_name};")
            print(f"Table '{table_name}' reordered successfully.")

        # Commit changes
        conn.commit()
        print("All tables processed.")
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    finally:
        conn.close()

reorder_all_tables(db_path, sort_column)


Processing table: pro_sup_data
Table 'pro_sup_data' reordered successfully.
All tables processed.
